In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [5]:
#fire module을 인스턴스화 할 수 있도록
def fire(x, squeeze, expand):
  #수축
  y = tf.keras.layers.Conv2D(filters=squeeze, kernel_size=1,
                             activation='relu', padding='same')(x)
  y = tf.keras.layers.BatchNormalization()(y)

  #팽창
  #y1 -> 1x1 conv
  y1 = tf.keras.layers.Conv2D(filters=expand//2, kernel_size=1,
                              activation='relu', padding='same')(y)
  y1 = tf.keras.layers.BatchNormalization()(y1)
  #y3 -> 3x3 conv
  y3 = tf.keras.layers.Conv2D(filters=expand//2, kernel_size=3,
                              activation = 'relu', padding='same')(y)
  y3 = tf.keras.layers.BatchNormalization()(y3)
  return tf.keras.layers.concatenate([y1, y3]) #1x1 conv와 3x3 conv의 결과를 이어 붙이기

In [8]:
#입력 x와 함께 호출되는 컨볼루션 레이어를 인스턴스화
def fire_module(squeeze, expand):
  return lambda x: fire(x, squeeze, expand)

In [10]:
#24 layer SqueezeNet
IMG_HEIGHT = 224
IMG_WIDTH = 224
CLASSES = ['flower1', 'flower2', 'flower3', 'flower4', 'flower5']


x = tf.keras.layers.Input(shape=[IMG_HEIGHT, IMG_WIDTH, 3])
y = tf.keras.layers.Conv2D(kernel_size=3, filters=32, padding='same', activation='relu')(x)
y = tf.keras.layers.BatchNormalization()(y)
y = fire_module(16, 32)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(48, 96)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(64, 128)(y)
y = fire_module(80, 160)(y)
y = fire_module(96, 192)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(112, 224)(y)
y = fire_module(128, 256)(y)
y = fire_module(160, 320)(y)
y = tf.keras.layers.MaxPooling2D(pool_size = 2)(y)
y = fire_module(192, 384)(y)
y = fire_module(224, 448)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(256, 512)(y)
y = tf.keras.layers.GlobalAveragePooling2D()(y)
y = tf.keras.layers.Dense(len(CLASSES), activation = 'softmax')(y)

model = tf.keras.Model(x, y)

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_27 (Conv2D)          (None, 224, 224, 32)         896       ['input_4[0][0]']             
                                                                                                  
 batch_normalization_27 (Ba  (None, 224, 224, 32)         128       ['conv2d_27[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 conv2d_28 (Conv2D)          (None, 224, 224, 16)         528       ['batch_normalization_27[0